In [66]:
import numpy as np
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
import tensorflow as tf
import tensorflow_quantum as tfq
from hep_VQAE import data_preprocessing as dp
from hep_VQAE import CAE as cae
import h5py
import keras
import numpy as np
from tensorflow.keras.utils import Sequence

In [67]:
f3 = h5py.File("../../data/quark-gluon/quark-gluon_train-set_n793900.hdf5","r")
f2 = h5py.File("../../data/quark-gluon/quark-gluon_test-set_n10000.hdf5","r")
f= h5py.File("../../data/quark-gluon/quark-gluon_test-set_n139306.hdf5","r")

In [ ]:
x_train = f.get('X_jets')
y_train = f.get('y')

x_val = f2.get('X')
y_val = f2.get('y')

x_test = f2.get('X')
y_test = f2.get('y')

In [ ]:
x_val_zeros = x_val[y_val[()]==0]
divide = np.sum(x_val_zeros, axis=(1,2)).reshape(x_val_zeros.shape[0],1,1,3)
divide[divide == 0] = 1
x_val_zeros = x_val_zeros/divide

x_test = x_val / divide

In [76]:
batch_size = 1990
num_batches = x_train.shape[0]//batch_size

In [ ]:
#maxx = 0
#minn = 10000000

#for i in range(int(num_batches)):
#    y = y_train[i * batch_size: (i + 1) * batch_size]
#    cmax = np.max(np.log(x_train[i * batch_size: (i + 1) * batch_size][y==0] + 1))
#    if cmax > maxx:
#        maxx = cmax
#    cmin = np.min(np.log(x_train[i * batch_size: (i + 1) * batch_size][y==0] + 1) )
#    if cmin < minn:
#        minn = cmin
#    print(i, end="\r")
#    
#print(maxx)
#print(minn)

In [79]:
from sklearn.preprocessing import normalize

class Generator(Sequence) :
  
    def __init__(self, hdf5_file, y_hdf5_file, batch_size, y_train_selection) :
        self.hdf5_file = hdf5_file
        self.batch_size = batch_size
        self.y_train_selection = y_train_selection
        self.y_hdf5_file = y_hdf5_file
    
  # Länge der Trainingsdaten (Anzahl der Batches)
    def __len__(self) :
        return x_train.shape[0]//self.batch_size
  
    def __getitem__(self, idx):
        y = self.y_hdf5_file[idx * self.batch_size: (idx + 1) * self.batch_size]
        x = self.hdf5_file[idx * self.batch_size: (idx + 1) * self.batch_size][y==self.y_train_selection]
        divide = np.sum(x, axis=(1,2)).reshape((x.shape[0],1,1,3))
        divide[divide == 0] = 1
        return x/divide, x/divide

In [80]:
traingen = Generator(x_train, y_train, 1900, 0)

In [81]:
model = cae.Convolutional_Autoencoder2(256)

In [82]:
model.encoder.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 42, 42, 64)        1792      
                                                                 
 conv2d_11 (Conv2D)          (None, 20, 20, 32)        18464     
                                                                 
 conv2d_12 (Conv2D)          (None, 10, 10, 30)        8670      
                                                                 
 conv2d_13 (Conv2D)          (None, 5, 5, 10)          2710      
                                                                 
Total params: 31,636
Trainable params: 31,636
Non-trainable params: 0
_________________________________________________________________


In [83]:
#model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.05))
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.0001))

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-7)
hist = model.fit_generator(traingen,epochs=30,validation_data=(x_val_zeros,x_val_zeros))

/tmp/ipykernel_276337/1952941689.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(traingen,epochs=30,validation_data=(x_val_zeros,x_val_zeros))


Epoch 1/30
43/73 [================>.............] - ETA: 17:56 - loss: 0.2481

In [ ]:
plt.plot(hist.history['loss'], label="training")
plt.plot(hist.history['val_loss'], label="validation")
plt.yscale('log')
plt.legend()

In [ ]:
def mae(x):
    pred = model.predict(x)
    return np.mean(np.abs(x - pred)**2,axis=(1,2,3))

def recon_acc(x):
    return 1 - mae(x)

In [ ]:
zerrooo = recon_acc(x_test[:2000])
one = recon_acc(x_test[8000:10000])

In [ ]:
print(np.median(zerrooo))
bins = np.histogram(np.hstack((zerrooo, one)), bins=100)[1]
#print(bins)
bins = np.arange(0.999995,0.9999955,0.00000005)
plt.hist(zerrooo, histtype='step', label="zero",bins=bins)#,bins=[0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.001,0.0011,0.0012,0.0013])
print(np.median(one))
plt.hist(one, histtype='step', label="one",bins=bins)#,bins=[0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.001,0.0011,0.0012,0.0013])
plt.xlabel("Reconstruction error")
plt.legend()

In [ ]:
def acc_for_threshold(T):
    # sixes that are predicted six
    num_six_right = np.sum(zerrooo > T)
    # threes that are predicted three
    num_three_right = np.sum(one < T)
    acc = (num_six_right + num_three_right)/(len(zerrooo) + len(one))
    return acc

Ts = np.linspace(0.999938,0.999939,1000)
accs = []
for i in Ts:
    accs.append(acc_for_threshold(i))
print(max(accs))
plt.xlabel("Anomaly threshold")
plt.ylabel("Acc.")
plt.plot(Ts, accs)  

In [ ]:
model.encoder(x_val_zeros[:10]).numpy()

In [ ]:
testimg = model((x_test[20]).reshape((1,125,125,3)))

In [ ]:
testimg[testimg>0]

In [ ]:
from matplotlib.colors import LogNorm
plt.imshow(testimg[0,:,:,2],cmap='binary',norm=LogNorm())
plt.colorbar()

In [ ]:
plt.imshow(x_test[20,:,:,2]+1,cmap='binary',norm=LogNorm())
plt.colorbar()

In [ ]:
plt.imshow(x_test[20,:,:,1]/0.07,cmap='binary')
plt.colorbar()